# code for generating and saving invoices
along with corresponding target arrays and truth values


This is the second version that still holds "nested csv's as serialized strings", but now we know how to eval() those into the original objects again later....

In [61]:
# remember to git pull the latest templates from 
# https://github.com/riklmr/MLiFC_invoice_classifier
# if you have not already (where else did this notebook come from?)

In [1]:
!ls

Baseline Ortec.ipynb		Jannes_Baseline_Ortec.ipynb  README.md
IBAN_generation.ipynb		LICENSE			     RegExDragon.ipynb
IBAN_stacker_274k.h5		ortec.py		     templates
IBAN_stacker.ipynb		Ortec_rik_1G.ipynb	     train
invoices_from_disk_demo.ipynb	Ortec_shaffy_1G.ipynb	     train_G_iban
invoices_generate_todisk.ipynb	ortec_templates_updated.zip
invoices_train.zipNOT		__pycache__


In [2]:
 !cat templates/TEMPLATE_18.txt

{'/Title': IndirectObject(49, 0), '/Author': IndirectObject(51, 0), '/Subject': IndirectObject(52, 0), '/Producer': IndirectObject(50, 0), '/Creator': IndirectObject(53, 0), '/CreationDate': IndirectObject(54, 0), '/ModDate': IndirectObject(54, 0), '/Keywords': IndirectObject(55, 0), '/AAPL:Keywords': IndirectObject(56, 0)}

            <SENDER_NAME>
V.O.F
                                        Specialist in tegelvlakke cementdekvloeren
                 Fam de Rochebrune
              Vaarsdrift 3
         Juinen
              Papendrecht
02-06-2017     factuur 17068
    -            Geachte heer,
                      Aan u geleverd cementdekvloer
45m27cmvezel bewapend
Bedrag excl BTW
900,00
!
 B.T.W 21%
189,00
!
 Totaal door u te voldoen
1.089,00
!
                                                                    Betalingen via IBAN.<IBAN>
binnen 14 dagen
 na datum factuur
     Dennehof 23
 3355RJ Papendrecht
 Mob.tel.0
6-53190711 IBAN.43ABNA.042.97.26082
 K.v.K <KVKNR>
72 BTW .nr

In [3]:
# this trick allows you to change underlying libraries on the fly

%reload_ext autoreload
%autoreload 2   # The 2 is important Rik!


In [4]:
import templates.invoicegen

import numpy as np
import pandas as pd
import glob
# import uuid


In [6]:
'''
Ignore:           0
Sender Name:      1 
Sender KVK:       2 
Sender IBAN:      3 
Invoice Reference:4
Total:            5
'''

train_dir = "train/"
! mkdir -p {train_dir}


In [8]:
# Create plenty of invoices for each template
templates_all = glob.glob("./templates/TEMPLATE_*.txt")
num_templates = len(templates_all)
PLENTY = 100000 // num_templates
print("Creating {} invoices times {} templates = {} in total".format(PLENTY, num_templates, PLENTY * num_templates))
print("Saving in directory {} ...".format(train_dir))

for txtname in templates_all:
    print("from template {}".format(txtname))
    with open(txtname, "r") as content_file:
        content = content_file.read()
        # Create invoices from template
        for i in range(PLENTY):
            # create_content() returns [invoice, target, truth]
            # invoice is string
            # target is array of classlabels per char in invoice
            # truth is array [uuid, SenderName, SenderKVK, SenderIBAN, Reference, Total]
            invoice, target, truth = templates.invoicegen.create_invoice(content)

            mysample = pd.DataFrame(columns = ['invoice', 'target', 'truth'])
            mysample.loc[0, 'invoice'] = invoice       # invoice already is a string
            mysample.loc[0, 'target']  = str(target)   # serialize array here
            mysample.loc[0, 'truth']   = str(truth)    # serialize array here

            uuid = truth[0]
            fname = train_dir + uuid + '.csv'
            # print("saving {} ...".format(fname))
            mysample.to_csv(fname, index=False)
        #
    #
#
print("done")


Creating 4761 invoices times 21 templates = 99981 in total
Saving in directory train/ ...
from template ./templates/TEMPLATE_1.txt
from template ./templates/TEMPLATE_10.txt
from template ./templates/TEMPLATE_8.txt
from template ./templates/TEMPLATE_2.txt
from template ./templates/TEMPLATE_9.txt
from template ./templates/TEMPLATE_20.txt
from template ./templates/TEMPLATE_4.txt
from template ./templates/TEMPLATE_18.txt
from template ./templates/TEMPLATE_17.txt
from template ./templates/TEMPLATE_11.txt
from template ./templates/TEMPLATE_5.txt
from template ./templates/TEMPLATE_7.txt
from template ./templates/TEMPLATE_14.txt
from template ./templates/TEMPLATE_13.txt
from template ./templates/TEMPLATE_19.txt
from template ./templates/TEMPLATE_12.txt
from template ./templates/TEMPLATE_3.txt
from template ./templates/TEMPLATE_16.txt
from template ./templates/TEMPLATE_21.txt
from template ./templates/TEMPLATE_6.txt
from template ./templates/TEMPLATE_15.txt
done


In [14]:
# ! rm -r {train_dir}
! du -sh {train_dir}

666M	train/


In [15]:
! zip -qr invoices_train.zip -s50m  ./train

In [16]:
! du -sh invoices_train*

50M	invoices_train.z01
50M	invoices_train.z02
1.6M	invoices_train.zip
